In [1]:
# Import necessary modules
# glob allows unix style pathname pattern expansion
from glob import glob
# pydicom is the python dicom reader
import pydicom as dicom
import nibabel.nicom.csareader as csareader
import os
import pandas as pd

/home/applications/spack/applications/gcc-8.2.0/miniconda3-4.5.11-oqs2mbgv3mmo3dll2f2rbxt4plfgyqzv/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  


ModuleNotFoundError: No module named 'pandas'

In [5]:
# 5-T1w_MPR_vNav -- MPRAGE structrual dicoms
# 6-fMRI_DistortionMap_PA -- fmri field map dicoms
# 7-fMRI_DistortionMap_AP -- fmri field map dicoms opposite phase encode direction
# 9-fMRI_REVL_ROI_loc_2 -- fmri localizer dicoms run1
# 10-fMRI_REVL_Study_1 -- fmri task dicoms run1
# 14-dMRI_DistortionMap_AP_dMRI_REVL -- dwi field map dicoms
# 16-dMRI_AP_REVL -- diffusion weighted dicoms

# directories will have to be specific to your computer
base_dir = f'/home/nmatt015/Mattfeld_PSB6351'
proj_dir = '/sourcedata'
subj_dir = '/Mattfeld_REVL-000-vCAT-021-S1/scans/10-fMRI_REVL_Study_1/resources/DICOM/files'

# with strings I can concatenate them with simple addition lines
data_dir = base_dir + proj_dir + subj_dir

# Here I use glob to grab the dicom files
# Why would I use glob?  What does it give me? #because it will let you search for files recursively
data_files = glob(data_dir + '/*')


In [6]:
data_files

['/home/nmatt015/Mattfeld_PSB6351/sourcedata/Mattfeld_REVL-000-vCAT-021-S1/scans/10-fMRI_REVL_Study_1/resources/DICOM/files/1.3.12.2.1107.5.2.43.166003.30000019061912563618000000017-10-287-8mzh2u.dcm',
 '/home/nmatt015/Mattfeld_PSB6351/sourcedata/Mattfeld_REVL-000-vCAT-021-S1/scans/10-fMRI_REVL_Study_1/resources/DICOM/files/1.3.12.2.1107.5.2.43.166003.30000019061912563618000000017-10-333-1yjps7s.dcm',
 '/home/nmatt015/Mattfeld_PSB6351/sourcedata/Mattfeld_REVL-000-vCAT-021-S1/scans/10-fMRI_REVL_Study_1/resources/DICOM/files/1.3.12.2.1107.5.2.43.166003.30000019061912563618000000017-10-244-123xmvy.dcm',
 '/home/nmatt015/Mattfeld_PSB6351/sourcedata/Mattfeld_REVL-000-vCAT-021-S1/scans/10-fMRI_REVL_Study_1/resources/DICOM/files/1.3.12.2.1107.5.2.43.166003.30000019061912563618000000017-10-298-b0qn24.dcm',
 '/home/nmatt015/Mattfeld_PSB6351/sourcedata/Mattfeld_REVL-000-vCAT-021-S1/scans/10-fMRI_REVL_Study_1/resources/DICOM/files/1.3.12.2.1107.5.2.43.166003.30000019061912563618000000017-10-92-1h

In [21]:
# Here I am using the python dicom reader to read in a dicom header
# What should go between the square brackets []
data_set = dicom.dcmread(data_files[0])

In [13]:
data_set

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 204
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.2.43.166003.2019061915561471881293473
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.40.0.13.1.1
(0002, 0013) Implementation Version Name         SH: 'dcm4che-2.0'
(0002, 0016) Source Application Entity Title     AE: 'AN_MEDCOMNT204'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M', 'ND', 'MOSAIC']
(0008, 0012) Instance Creation Date              DA: '20190619'
(0008, 0013) Instance Creation Time              TM: '155635.410000'
(0008, 0016) SOP Cl

In [82]:
## Elements we're interested in
# Repetition Time = 1760.0
# Echo Time = 35.0
# Acquisition Matrix = [100, 0, 0, 90]
# Flip Angle = 52.0
# Acquisition Number = 287
# Protocol Name = fMRI_REVL_Study_1
# Slice times = data_set[0x00191029].value

print('The number of TRs collected equals: {0}'.format(data_set[0x00180080].value))
print('The timing of the TR was: {0} ms'.format(data_set[0x00180080].value))
print('The timing of the TE was: {0} ms'.format(data_set[0x00180081].value))
print('The flip angle in degrees was: {0}'.format(data_set[0x00181312].value))
print(f'The name of the scan was: {"fMRI_REVL_Study_1"}')
print(f'The phase encode direction was: {"COL"}')
print('COL = AP or PA; ROW = RL or LR')
print('The slice timing was: {0}'.format(data_set[0x00191029].value))
print('The field of view - FOV was: {0}'.format(data_set[0x00180094].value))

# Why use the following format to access information in the header?
#print(data_set[int('0051100c', 16)].value) # data_set[0x0051100c].value
# Allows us to access information that is private in the dicom header.

# How do I find phase encoding direction information?

csa_str = data_set[int('00291010', 16)].value
csa_tr = csareader.read(csa_str)
print('The phase encode direction was: {0}'.format(csa_str[1]))
print('1 = +; 0 = -')
print('COL, 1 = PA; COL, 0 = AP')

The number of TRs collected equals: 1760
The timing of the TR was: 1760 ms
The timing of the TE was: 35 ms
The flip angle in degrees was: COL
The name of the scan was: fMRI_REVL_Study_1
The phase encode direction was: COL
COL = AP or PA; ROW = RL or LR
The slice timing was: [1265.00000001, 0.0, 870.0, 77.50000001, 947.50000001, 157.5, 1027.5, 237.50000002, 1107.50000001, 315.0, 1185.0, 475.00000001, 1345.0, 552.50000002, 1422.50000002, 632.50000001, 1502.5, 709.99999999, 1580.00000002, 790.00000001, 1660.0, 395.00000002, 1265.00000001, 0.0, 870.0, 77.50000001, 947.50000001, 157.5, 1027.5, 237.50000002, 1107.50000001, 315.0, 1185.0, 475.00000001, 1345.0, 552.50000002, 1422.50000002, 632.50000001, 1502.5, 709.99999999, 1580.00000002, 790.00000001, 1660.0, 395.00000002, 1265.00000001, 0.0, 870.0, 77.50000001, 947.50000001, 157.5, 1027.5, 237.50000002, 1107.50000001, 315.0, 1185.0, 475.00000001, 1345.0, 552.50000002, 1422.50000002, 632.50000001, 1502.5, 709.99999999, 1580.00000002, 790.000

In [77]:
csa_str = data_set[int('00291010', 16)].value
csa_tr = csareader.read(csa_str)

In [59]:
csa_tr


{'tags': {'EchoLinePosition': {'n_items': 6,
   'vm': 1,
   'vr': 'IS',
   'syngodt': 6,
   'last3': 77,
   'tag_no': 0,
   'items': [45]},
  'EchoColumnPosition': {'n_items': 6,
   'vm': 1,
   'vr': 'IS',
   'syngodt': 6,
   'last3': 77,
   'tag_no': 1,
   'items': [50]},
  'EchoPartitionPosition': {'n_items': 6,
   'vm': 1,
   'vr': 'IS',
   'syngodt': 6,
   'last3': 77,
   'tag_no': 2,
   'items': [32]},
  'UsedChannelMask': {'n_items': 0,
   'vm': 1,
   'vr': 'UL',
   'syngodt': 9,
   'last3': 205,
   'tag_no': 3,
   'items': []},
  'UsedChannelString': {'n_items': 6,
   'vm': 1,
   'vr': 'UT',
   'syngodt': 27,
   'last3': 77,
   'tag_no': 4,
   'items': ['XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX']},
  'Actual3DImaPartNumber': {'n_items': 0,
   'vm': 1,
   'vr': 'IS',
   'syngodt': 6,
   'last3': 205,
   'tag_no': 5,
   'items': []},
  'ICE_Dims': {'n_items': 6,
   'vm': 1,
   'vr': 'LO',
   'syngodt': 19,
   'last3': 77,
   'tag_no': 6,
   'items': ['X_1_1_1_287_1_1_34_1_1_1_1_183']},
  

b'SV10\x04\x03\x02\x01e\x00\x00\x00M\x00\x00\x00EchoLinePosition\x00\x10*\x12\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00IS\x00\x00\x06\x00\x00\x00\x06\x00\x00\x00M\x00\x00\x00\t\x00\x00\x00\t\x00\x00\x00M\x00\x00\x00\t\x00\x00\x0045      \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xcd\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xcd\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xcd\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xcd\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xcd\x00\x00\x00\x00\x00\x00\x00EchoColumnPosition\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00y"\x01\x00\x00\x00IS\x00\x00\x06\x00\x00\x00\x06\x00\x00\x00M\x00\x00\x00\t\x00\x00